In [ ]:
// 无变量 prompt
import { PromptTemplate } from "@langchain/core/prompts";

const greetingPrompt = new PromptTemplate({
  inputVariables: [],
  template: 'hello world',
});
const formattedGreetingPrompt = await greetingPrompt.format();

console.log(formattedGreetingPrompt);

In [ ]:
// 含变量的 template
import { PromptTemplate } from "@langchain/core/prompts";
const prompt = new PromptTemplate({
  inputVariables: ['name', 'word'],
  // {} 需要转义
  template: 'hello-{word} {name}, {{test}}',
});

const formattedPrompt = await prompt.format({
  word: 'everyone',
  name: 'NanluQingshi',
});

console.log(formattedPrompt);

In [ ]:
// 快捷方式
import { PromptTemplate } from "@langchain/core/prompts";
const prompt = PromptTemplate.fromTemplate('hello-{word} {name}, {{test}}',
);

const formattedPrompt = await prompt.format({
  word: 'everyone',
  name: 'NanluQingshi',
});

console.log(formattedPrompt);

In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";
const initalPrompt = new PromptTemplate({
  template: "这是一个{type}，它是{item}。",
  inputVariables: ["type", "item"],
});

const partialedPrompt = await initalPrompt.partial({
  type: 'list',
});

const fp1 = await partialedPrompt.format({
  item: 'value',
});

console.log(fp1);

const fp2 = await partialedPrompt.format({
  item: 'key',
});

console.log(fp2);

In [ ]:
import { PromptTemplate } from "@langchain/core/prompts";
// 动态填充参数
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

function generateGreeting(timeOfDay) {
  return () => {
    const date = getCurrentDateStr()
    switch (timeOfDay) {
      case 'morning':
        return date + ' 早上好';
      case 'afternoon':
        return date + ' 下午好';
      case 'evening':
        return date + ' 晚上好';
      default:
        return date + ' 你好';
    }
  };
};

const prompt = new PromptTemplate({
  template: '{greeting}',
  inputVariables: ['greeting'],
});

const currentTimeOfDay = 'afternoon';
const partialPrompt = await prompt.partial({
  greeting: generateGreeting(currentTimeOfDay),
});

// 不支持在 format 参数中的函数传参
const formattedPrompt = await partialPrompt.format();
console.log(formattedPrompt);


In [ ]:
// chat prompt
import { SystemMessagePromptTemplate } from "@langchain/core/prompts";
import { HumanMessagePromptTemplate } from "@langchain/core/prompts";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const systemTemplate = "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。";
const humanTemplate = "请翻译这句话：{text}";

// const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(systemTemplate);
// const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate(humanTemplate)
// const chatPrompt = ChatPromptTemplate.fromMessages([
//   translateInstructionTemplate,
//   userQuestionTemplate,
// ]);

const chatPrompt = ChatPromptTemplate.fromMessages([
  ['system', systemTemplate],
  ['human', humanTemplate],
]);

const formattedChatPrompt = await chatPrompt.formatMessages({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
});

console.log(formattedChatPrompt);

In [ ]:
import { Ollama } from '@langchain/community/llms/ollama';

const llm = new Ollama({ 
  baseUrl: 'http://localhost:11434',
  model: 'llama3.1:8b',
});

const chain = chatPrompt.pipe(llm);
const res = await chain.invoke({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
});

console.log(res);

In [ ]:
// 组合多个 Prompt 
import {
  PromptTemplate,
  PipelinePromptTemplate,
} from "@langchain/core/prompts";

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const fullPrompt = PromptTemplate.fromTemplate(
  `你是一个智能管家，今天是 {date}，你的主人的信息是{info}，
  根据上下文，完成主人的需求
  {task}`
);
const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}");
const periodPrompt = await datePrompt.partial({
  date: getCurrentDateStr,
});

const infoPrompt = PromptTemplate.fromTemplate('姓名是{name}，性别是{gender}');
const taskPrompt = PromptTemplate.fromTemplate(
  `我想吃 {period} 的 {food}。 
  再重复一遍我的信息 {info}`
);

const composedPrompt = new PipelinePromptTemplate({
  pipelinePrompts: [
    {
      name: 'date',
      prompt: periodPrompt,
    },
    {
      name: 'info',
      prompt: infoPrompt,
    },
    {
      name: 'task',
      prompt: taskPrompt,
    }
  ],
  finalPrompt: fullPrompt,
});

const formattedPrompt = await composedPrompt.format({
  period: '晚上',
  name: '甄姬',
  gender: 'female',
  food: 'watermelon',
});

console.log(formattedPrompt);


你是一个智能管家，今天是 2/4/2026，现在是 晚上，你的主人的信息是姓名是甄姬，性别是female，
  根据上下文，完成主人的需求
  我想吃 晚上 的 watermelon。 
  再重复一遍我的信息 姓名是甄姬，性别是female
